<a href="https://colab.research.google.com/github/Malaha-SOUMARE/Exo2_Pushing/blob/main/projet_Walmart_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importer des bibliothèques

In [238]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display

Exploitation du fichier

In [239]:
data = pd.read_csv('/content/Walmart_Store_sales.csv')
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [240]:
data.describe

<bound method NDFrame.describe of      Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      6.0  18-02-2011    1572117.54           NaN        59.61       3.045   
1     13.0  25-03-2011    1807545.43           0.0        42.38       3.435   
2     17.0  27-07-2012           NaN           0.0          NaN         NaN   
3     11.0         NaN    1244390.03           0.0        84.57         NaN   
4      6.0  28-05-2010    1644470.66           0.0        78.89       2.759   
..     ...         ...           ...           ...          ...         ...   
145   14.0  18-06-2010    2248645.59           0.0        72.62       2.780   
146    7.0         NaN     716388.81           NaN        20.74       2.778   
147   17.0  11-06-2010     845252.21           0.0        57.14       2.841   
148    8.0  12-08-2011     856796.10           0.0        86.05       3.638   
149   19.0  20-04-2012    1255087.26           0.0        55.20       4.170   

            CPI  Unemployment  
0    214.777523         6.858  
1    128.616064         7.470  
2    130.719581         5.936  
3    214.556497         7.346  
4    212.412888         7.092  
..          ...           ...  
145  182.442420         8.899  
146         NaN           NaN  
147  126.111903           NaN  
148  219.007525           NaN  
149  137.923067         8.150  

[150 rows x 8 columns]>

In [241]:
data.shape

(150, 8)

Prétraitement EDA

In [242]:
data = pd.DataFrame(data)

# Supprimer les lignes où les valeurs cibles sont manquantes
data_sans_valeurs_manquantes = data.dropna(subset=['Weekly_Sales'])

# Afficher le DataFrame résultant
print(data_sans_valeurs_manquantes)

     Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      6.0  18-02-2011    1572117.54           NaN        59.61       3.045   
1     13.0  25-03-2011    1807545.43           0.0        42.38       3.435   
3     11.0         NaN    1244390.03           0.0        84.57         NaN   
4      6.0  28-05-2010    1644470.66           0.0        78.89       2.759   
5      4.0  28-05-2010    1857533.70           0.0          NaN       2.756   
..     ...         ...           ...           ...          ...         ...   
145   14.0  18-06-2010    2248645.59           0.0        72.62       2.780   
146    7.0         NaN     716388.81           NaN        20.74       2.778   
147   17.0  11-06-2010     845252.21           0.0        57.14       2.841   
148    8.0  12-08-2011     856796.10           0.0        86.05       3.638   
149   19.0  20-04-2012    1255087.26           0.0        55.20       4.170   

            CPI  Unemployment  
0    214.777523    

In [243]:
data_desc = data.describe(include='all')
data_desc

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [244]:
# Supprimer les lignes avec des valeurs manquantes dans la colonne "Weekly_Sales"
data = data.dropna(subset=['Weekly_Sales'])

# Afficher le DataFrame résultant
print(data)

     Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      6.0  18-02-2011    1572117.54           NaN        59.61       3.045   
1     13.0  25-03-2011    1807545.43           0.0        42.38       3.435   
3     11.0         NaN    1244390.03           0.0        84.57         NaN   
4      6.0  28-05-2010    1644470.66           0.0        78.89       2.759   
5      4.0  28-05-2010    1857533.70           0.0          NaN       2.756   
..     ...         ...           ...           ...          ...         ...   
145   14.0  18-06-2010    2248645.59           0.0        72.62       2.780   
146    7.0         NaN     716388.81           NaN        20.74       2.778   
147   17.0  11-06-2010     845252.21           0.0        57.14       2.841   
148    8.0  12-08-2011     856796.10           0.0        86.05       3.638   
149   19.0  20-04-2012    1255087.26           0.0        55.20       4.170   

            CPI  Unemployment  
0    214.777523    

In [245]:
# Convertir la colonne "Date" au format de date/heure si nécessaire
data['Date'] = pd.to_datetime(data['Date'])

# Créer les nouvelles fonctionnalités à partir de la colonne "Date"
data['année'] = data['Date'].dt.year
data['mois'] = data['Date'].dt.month
data['jour'] = data['Date'].dt.day
data['jour_de_la_semaine'] = data['Date'].dt.dayofweek

# Afficher le DataFrame avec les nouvelles fonctionnalités
print(data)

     Store       Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      6.0 2011-02-18    1572117.54           NaN        59.61       3.045   
1     13.0 2011-03-25    1807545.43           0.0        42.38       3.435   
3     11.0        NaT    1244390.03           0.0        84.57         NaN   
4      6.0 2010-05-28    1644470.66           0.0        78.89       2.759   
5      4.0 2010-05-28    1857533.70           0.0          NaN       2.756   
..     ...        ...           ...           ...          ...         ...   
145   14.0 2010-06-18    2248645.59           0.0        72.62       2.780   
146    7.0        NaT     716388.81           NaN        20.74       2.778   
147   17.0 2010-06-11     845252.21           0.0        57.14       2.841   
148    8.0 2011-08-12     856796.10           0.0        86.05       3.638   
149   19.0 2012-04-20    1255087.26           0.0        55.20       4.170   

            CPI  Unemployment   année  mois  jour  jour_de_la_s

<ipython-input-245-1ef7b9d313ec>:2: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [246]:
from typing_extensions import dataclass_transform
# Calculer la moyenne et l'écart type pour chaque caractéristique numérique
mean = data.mean()
std = data.std()

# Définir la plage acceptable pour chaque caractéristique numérique
lower_bound = mean - 3 * std
upper_bound = mean + 3 * std

# Filtrer le DataFrame en fonction des plages spécifiées
for column in data.select_dtypes(include='number'):
    dataclass_transform = data[(data[column] >= lower_bound[column]) & (data[column] <= upper_bound[column])]

# Afficher le DataFrame résultant
print(data)

     Store       Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      6.0 2011-02-18    1572117.54           NaN        59.61       3.045   
1     13.0 2011-03-25    1807545.43           0.0        42.38       3.435   
3     11.0        NaT    1244390.03           0.0        84.57         NaN   
4      6.0 2010-05-28    1644470.66           0.0        78.89       2.759   
5      4.0 2010-05-28    1857533.70           0.0          NaN       2.756   
..     ...        ...           ...           ...          ...         ...   
145   14.0 2010-06-18    2248645.59           0.0        72.62       2.780   
146    7.0        NaT     716388.81           NaN        20.74       2.778   
147   17.0 2010-06-11     845252.21           0.0        57.14       2.841   
148    8.0 2011-08-12     856796.10           0.0        86.05       3.638   
149   19.0 2012-04-20    1255087.26           0.0        55.20       4.170   

            CPI  Unemployment   année  mois  jour  jour_de_la_s

In [247]:
# Sélectionner la colonne de la variable cible (Y)
Y = data['Weekly_Sales']

# Sélectionner les autres colonnes représentant les caractéristiques (X)
X = data.drop(columns=['Weekly_Sales'])

# Afficher la variable cible (Y)
print(Y)

# Afficher les caractéristiques (X)
print(X)

0      1572117.54
1      1807545.43
3      1244390.03
4      1644470.66
5      1857533.70
          ...    
145    2248645.59
146     716388.81
147     845252.21
148     856796.10
149    1255087.26
Name: Weekly_Sales, Length: 136, dtype: float64
     Store       Date  Holiday_Flag  Temperature  Fuel_Price         CPI  \
0      6.0 2011-02-18           NaN        59.61       3.045  214.777523   
1     13.0 2011-03-25           0.0        42.38       3.435  128.616064   
3     11.0        NaT           0.0        84.57         NaN  214.556497   
4      6.0 2010-05-28           0.0        78.89       2.759  212.412888   
5      4.0 2010-05-28           0.0          NaN       2.756  126.160226   
..     ...        ...           ...          ...         ...         ...   
145   14.0 2010-06-18           0.0        72.62       2.780  182.442420   
146    7.0        NaT           NaN        20.74       2.778         NaN   
147   17.0 2010-06-11           0.0        57.14       2.841  126.1119

In [248]:
# Variables catégorielles
variables_catégorielles = ['Store', 'Holiday_Flag']

# Variables numériques
variables_numériques = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'année', 'mois', 'jour', 'jour_de_la_semaine']

# Sélectionner les colonnes catégorielles
X_catégorielles = data[variables_catégorielles]

# Sélectionner les colonnes numériques
X_numériques = data[variables_numériques]

# Afficher les variables catégorielles
print("Variables catégorielles :\n", X_catégorielles)

# Afficher les variables numériques
print("\nVariables numériques :\n", X_numériques)

Variables catégorielles :
      Store  Holiday_Flag
0      6.0           NaN
1     13.0           0.0
3     11.0           0.0
4      6.0           0.0
5      4.0           0.0
..     ...           ...
145   14.0           0.0
146    7.0           NaN
147   17.0           0.0
148    8.0           0.0
149   19.0           0.0

[136 rows x 2 columns]

Variables numériques :
      Temperature  Fuel_Price         CPI  Unemployment   année  mois  jour  \
0          59.61       3.045  214.777523         6.858  2011.0   2.0  18.0   
1          42.38       3.435  128.616064         7.470  2011.0   3.0  25.0   
3          84.57         NaN  214.556497         7.346     NaN   NaN   NaN   
4          78.89       2.759  212.412888         7.092  2010.0   5.0  28.0   
5            NaN       2.756  126.160226         7.896  2010.0   5.0  28.0   
..           ...         ...         ...           ...     ...   ...   ...   
145        72.62       2.780  182.442420         8.899  2010.0   6.0  18.0   


In [249]:
# Supprimer les lignes contenant des valeurs manquantes
data_sans_nan = data.dropna()

# Afficher le DataFrame résultant
print(data_sans_nan)

     Store       Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
1     13.0 2011-03-25    1807545.43           0.0        42.38       3.435   
4      6.0 2010-05-28    1644470.66           0.0        78.89       2.759   
6     15.0 2011-06-03     695396.19           0.0        69.80       4.069   
7     20.0 2012-02-03    2203523.20           0.0        39.93       3.617   
10     8.0 2011-08-19     895066.50           0.0        82.92       3.554   
..     ...        ...           ...           ...          ...         ...   
139    7.0 2012-05-25     532739.77           0.0        50.60       3.804   
143    3.0 2010-06-04     396968.80           0.0        78.53       2.705   
144    3.0 2012-10-19     424513.08           0.0        73.44       3.594   
145   14.0 2010-06-18    2248645.59           0.0        72.62       2.780   
149   19.0 2012-04-20    1255087.26           0.0        55.20       4.170   

            CPI  Unemployment   année  mois  jour  jour_de_la_s

Partie 2 : Modèle de base (régression linéaire)

In [250]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [251]:
import pandas as pd

# Supposez que votre DataFrame s'appelle df et que vous voulez convertir la colonne 'timestamp_column'

# Convertir la colonne 'timestamp_column' en type 'datetime'
df = pd.to_datetime

# Maintenant, vous pouvez utiliser cette colonne dans vos opérations sans rencontrer l'erreur


In [252]:
features_list = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'année', 'mois', 'jour', 'jour_de_la_semaine']
numeric_indices = [0]
categorical_indices = []
target_variable = ['Store', 'Holiday_Flag']

In [253]:
# Divide dataset Train set & Test set
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [254]:
cat_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'année', 'mois', 'jour', 'jour_de_la_semaine']
for i in range(len(cat_features)):
    fig = px.bar(data[cat_features[i]])
    fig.show()

In [255]:
# Correlation matrix
corr_matrix = data.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

In [256]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'année', 'mois', 'jour', 'jour_de_la_semaine']
target_variable = ['Store', 'Holiday_Flag']

X = data.loc[:,features_list]
Y = data.loc[:,target_variable]

print("...Done.")
print()
print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
   Store  Holiday_Flag
0    6.0           NaN
1   13.0           0.0
3   11.0           0.0
4    6.0           0.0
5    4.0           0.0

X :
   Temperature  Fuel_Price         CPI  Unemployment   année  mois  jour  \
0        59.61       3.045  214.777523         6.858  2011.0   2.0  18.0   
1        42.38       3.435  128.616064         7.470  2011.0   3.0  25.0   
3        84.57         NaN  214.556497         7.346     NaN   NaN   NaN   
4        78.89       2.759  212.412888         7.092  2010.0   5.0  28.0   
5          NaN       2.756  126.160226         7.896  2010.0   5.0  28.0   

   jour_de_la_semaine  
0                 4.0  
1                 4.0  
3                 NaN  
4                 4.0  
5                 4.0  


In [257]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'année', 'mois', 'jour', 'jour_de_la_semaine']
Found categorical features  []


In [258]:
# Divide dataset Train set & Test set
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



Prétraitement

In [259]:
# Create pipeline for numeric features
from sklearn.pipeline import Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [260]:
# Create pipeline for categorical features
from sklearn.preprocessing import OneHotEncoder
categorical_transformer = Pipeline(
    steps=[('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [261]:
from sklearn.compose import ColumnTransformer
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [262]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()

Performing preprocessings on train set...
     Temperature  Fuel_Price         CPI  Unemployment   année  mois  jour  \
118        82.99       2.637  215.016648         6.384  2010.0   6.0  18.0   
11         52.02       2.878  132.763355         9.331  2010.0  10.0  15.0   
3          84.57         NaN  214.556497         7.346     NaN   NaN   NaN   
54         86.87       3.666  130.719633         7.170  2012.0   7.0   6.0   
108        69.12       2.906  132.293936           NaN  2010.0   5.0  28.0   

     jour_de_la_semaine  
118                 4.0  
11                  4.0  
3                   NaN  
54                  4.0  
108                 4.0  
...Done.
[[ 1.20958854 -1.5030448   0.98174462 -0.77724804 -1.09299874 -0.08547043
   0.21592148  0.        ]
 [-0.58190959 -0.97839086 -1.12750195  0.99676768 -1.09299874  1.28205649
  -0.19319291  0.        ]
 [ 1.3009856   0.          0.96994481 -0.19814959  0.          0.
   0.          0.        ]
 [ 1.43403196  0.73707512 -1.

Modèle de train


In [263]:
# Supprimer les lignes avec des valeurs NaN dans la variable cible (y)
data = data.dropna(subset=['Weekly_Sales'])

# Assurez-vous de réinitialiser l'index après la suppression des lignes
data = data.reset_index(drop=True)


In [264]:
#Supprimer les colonnes avec des valeurs nulles
data_sans_nulles = data.dropna(axis=1)

# Afficher le DataFrame résultant
print(data_sans_nulles)

     Store  Weekly_Sales
0      6.0    1572117.54
1     13.0    1807545.43
2     11.0    1244390.03
3      6.0    1644470.66
4      4.0    1857533.70
..     ...           ...
131   14.0    2248645.59
132    7.0     716388.81
133   17.0     845252.21
134    8.0     856796.10
135   19.0    1255087.26

[136 rows x 2 columns]


In [265]:
# Supprimer les colonnes avec toutes les valeurs nulles
data_sans_nulles = data.dropna(axis=1, how='all')


In [266]:
# Supprimer les lignes où la variable cible (y) contient des valeurs NaN
data = data.dropna(subset=['Weekly_Sales'])

# Réinitialiser l'index après la suppression des lignes
data = data.reset_index(drop=True)

In [267]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...


ValueError: Input y contains NaN.